 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P 500 Index.

In [3]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path

%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp500_history.csv`: Contains historical closing prices of the S&P 500 Index.

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

In [7]:
# Reading whale returns:

whale_returns_data = Path("./Resources/whale_returns.csv")

whale_returns_df = pd.read_csv(whale_returns_data)


# Setup collumns:

whale_returns_df.columns = ["Date", "Soros_Fund", "Paulson_Fund", "Tiger_Global_Fund", "Berkshire_Fund"]

# Set CSV to a dataframe:

whale_returns_df.set_index(pd.to_datetime(whale_returns_df["Date"], infer_datetime_format = True), inplace = True)

# Drop dups and index the date column:

whale_returns_df.drop(columns = ["Date"], inplace = True)

# Check for nulls

print(f"Null values for Whale Returns:\n{whale_returns_df.isnull().sum()}")

Null values for Whale Returns:
Soros_Fund           1
Paulson_Fund         1
Tiger_Global_Fund    1
Berkshire_Fund       1
dtype: int64


In [5]:
#set index
sp500_history_df = sp500_history_df.set_index("date")
aapl_historical_df = aapl_historical_df.set_index("date")
cost_historical_df = cost_historical_df.set_index("date")
goog_historical_df = goog_historical_df.set_index("date")

# Sort each of the data frames with respect to their corresponding dates
sp500_history_df.sort_index(inplace=True)
aapl_historical_df.sort_index(inplace=True)
cost_historical_df.sort_index(inplace=True)
goog_historical_df.sort_index(inplace=True)

combined_df = pd.concat([sp500_history_df, aapl_historical_df, goog_historical_df, cost_historical_df], axis = "columns", join = "inner")

combined_df.columns = ['SP500', 'AAPL', 'COST', 'GOOG']

#sp500_history_df.head()
combined_df.head()

KeyError: "None of ['date'] are in the columns"

In [4]:
# Clean and assign date to drop any un-wanted dollar 
#drop the dollar signs
def clean_dollarsign(value):

    if isinstance(value, str):
        return(value.replace('$', ''))
    return(value)

In [3]:
#set column names
sp500_history_df.columns = ["date", "close"]
whale_returns_df.columns = ["date", "soros_fund", "paulson_fund", "tiger_global_fund", "berkshare_fund"]
algo_returns_df.columns = ["date", "algo_1", "algo_2"]

#set the date format from the csv file into the sp500 dataframe
sp500_history_df['date'] = pd.to_datetime(sp500_history_fd['date'], format='%d-%b-%y')

#remove the dollar sign and change thetypeto float
sp500_history_df['close'] = sp500_history_df['close'].apply(clean_dollar_sign).astype('float')

#set index
sp500_history_df = sp500_history_df.set_index("date")
whale_returns_df = whale_returns_df.set_index("date")
algo_returns_df = algo_returns_df.set_index("date")

# Sort each of the data frames with respect to their corresponding dates
sp500_history_df.sort_index(inplace=True)
whale_returns_df.sort_index(inplace=True)
algo_returns_df.sort_index(inplace=True)


#calculate returns for the sp500_history_df
sp500_returns_df = sp500_csv.pct_change()

#check for null values

print(f" Null values in s&p500 :\n{sp500_history_df.isnull().sum()}\n")
print(f" Null values in Whale :\n{whale_returns_df.isnull().sum()}\n")
print(f" Null values in Algo :\n{sp500_history_df.isnull().sum()}\n")


NameError: name 'whale_returns_df' is not defined

In [ ]:
# Print out all CSVs to see the drop dollar sign changes

print(sp500_history_df.head())
print(sp500_history_df.dtypes)

print(aapl_historical_df.head())
print(aapl_historical_df.dtypes)

print(cost_historical_df.head())
print(cost_historical_df.dtypes)

print(goog_historical_df.head())
print(goog_historical_df.dtypes)

In [ ]:
# Count nulls

sp500_history_df.isnull().sum()

aapl_historical_df.isnull().sum()

cost_historical_df.isnull().sum()

goog_historical_df.isnull().sum()

In [ ]:
# Drop nulls

sp500_history_df.dropna()

aapl_historical_df.dropna()

cost_historical_df.dropna()

goog_historical_df.dropna()

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [ ]:
# Reading algorithmic returns

algo_returns_data = Path("./Resources/algo_returns.csv")

algo_returns_df = pd.read_csv(algo_returns_data)

#set columns
algo_returns_df.columns = ['date', 'Algo1', 'Algo2',]

print(algo_returns_df.head(10))
print(algo_returns_df.dtypes)


In [3]:
# Count nulls
also_returns_data.isnull().sum()


NameError: name 'also_returns_data' is not defined

In [4]:
# Drop nulls
also_returns_data.dropna()

NameError: name 'also_returns_data' is not defined

## S&P 500 Returns

Read the S&P 500 historic closing prices and create a new daily returns DataFrame from the data. 

In [10]:
# Reading S&P 500 Closing Prices


In [11]:
# Check Data Types


In [12]:
# Fix Data Types


In [13]:
# Calculate Daily Returns


In [14]:
# Drop nulls


In [15]:
# Rename `Close` Column to be specific to this portfolio.


## Combine Whale, Algorithmic, and S&P 500 Returns

In [16]:
# Join Whale Returns, Algorithmic Returns, and the S&P 500 Returns into a single DataFrame with columns for each portfolio's returns.



---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

In [17]:
# Plot daily returns of all portfolios


#### Calculate and Plot cumulative returns.

In [18]:
# Calculate cumulative returns of all portfolios

# Plot cumulative returns


---

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

### Create a box plot for each portfolio


In [19]:
# Box plot to visually show risk


### Calculate Standard Deviations

In [20]:
# Calculate the daily standard deviations of all portfolios


### Determine which portfolios are riskier than the S&P 500

In [21]:
# Calculate  the daily standard deviation of S&P 500

# Determine which portfolios are riskier than the S&P 500


### Calculate the Annualized Standard Deviation

In [22]:
# Calculate the annualized standard deviation (252 trading days)


---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for the S&P 500 using a 21-day window
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P 500
3. Choose one portfolio, then calculate and plot the 60-day rolling beta between it and the S&P 500

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [23]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window

# Plot the rolling standard deviation


### Calculate and plot the correlation

In [24]:
# Calculate the correlation

# Display de correlation matrix


### Calculate and Plot Beta for a chosen portfolio and the S&P 500

In [25]:
# Calculate covariance of a single portfolio

# Calculate variance of S&P 500

# Computing beta

# Plot beta trend


## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half-life.

In [26]:
# Use `ewm` to calculate the rolling window


---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [27]:
# Annualized Sharpe Ratios


In [28]:
# Visualize the sharpe ratios as a bar plot


### Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.

Write your answer here!

---

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others
5. Include correlation analysis to determine which stocks (if any) are correlated

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

For this demo solution, we fetch data from three companies listes in the S&P 500 index.

* `GOOG` - [Google, LLC](https://en.wikipedia.org/wiki/Google)

* `AAPL` - [Apple Inc.](https://en.wikipedia.org/wiki/Apple_Inc.)

* `COST` - [Costco Wholesale Corporation](https://en.wikipedia.org/wiki/Costco)

In [29]:
# Reading data from 1st stock


In [30]:
# Reading data from 2nd stock


In [31]:
# Reading data from 3rd stock


In [32]:
# Combine all stocks in a single DataFrame


In [33]:
# Reset Date index


In [34]:
# Reorganize portfolio data by having a column per symbol


In [35]:
# Calculate daily returns

# Drop NAs

# Display sample data


## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [36]:
# Set weights
weights = [1/3, 1/3, 1/3]

# Calculate portfolio return

# Display sample data


## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [37]:
# Join your returns DataFrame to the original returns DataFrame


In [38]:
# Only compare dates where return data exists for all the stocks (drop NaNs)


## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [39]:
# Calculate the annualized `std`


### Calculate and plot rolling `std` with 21-day window

In [40]:
# Calculate rolling standard deviation

# Plot rolling standard deviation


### Calculate and plot the correlation

In [41]:
# Calculate and plot the correlation


### Calculate and Plot Rolling 60-day Beta for Your Portfolio compared to the S&P 500

In [42]:
# Calculate and plot Beta


### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [43]:
# Calculate Annualzied Sharpe Ratios


In [44]:
# Visualize the sharpe ratios as a bar plot


### How does your portfolio do?

Write your answer here!